# Data Downtime Challenge | Exercise 3

## 0. Setup

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from data.utils.exercise_3 import all_days

In [ ]:
import sqlite3

conn = sqlite3.connect("../data/dbs/Ex3.db")
c = conn.cursor()

## 1. Introduction

In the last exercise, we looked at incidents spanning multiple tables in a database. Yet, we've still only been looking at _individual_ metrics like the row count, rate of null values, and so on. In practice, many genuine data downtime incidents involve _conjunctions_ of events across multiple upstream and downstream tables. In this exercise, we practice crafting single queries that can handle such conjunctive events.

## 2. Data

In this exercise, we'll continue to use the `EXOPLANETS`, `HABITABLES`, and `EXOPLANETS_SCHEMA` tables. There is also a `HABITABLES_SCHEMA` table, and all four contain additional entries beyond those in exercise 2.

In [ ]:
# show all tables in DB
pd.read_sql_query("""
    SELECT 
        name
    FROM 
        sqlite_master 
    WHERE 
        type ='table' AND 
        name NOT LIKE 'sqlite_%';
    """,
    conn
)

## 3. Exercise: Pillars of Data Observability in Conjunction

Why care about conjunctions of events, when individual events provide all the information you need? One important factor is **noise** -- looking at simultaneous events reduces the total number of events you're worried about, and makes it more lilely that the issues are genuine. Another factor is **causality** -- given an issue in some table, looking at simultaneous events in upstream tables might help you determine the root cause, and quicken the path to a solution.

Take the past exercise as an example -- we saw that the `habitability` field had anomalous rates of zeroed values:

In [ ]:
h_zero = pd.read_sql_query("""
SELECT
    DATE_ADDED,
    CAST(SUM(CASE WHEN HABITABILITY IS 0 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) AS ZERO_RATE
FROM
    HABITABLES
GROUP BY
    DATE_ADDED
""", conn)

h_zero = h_zero \
    .rename(columns={clmn: clmn.lower() for clmn in h_zero.columns}) \
    .set_index("date_added") \
    .reindex(all_days)

In [ ]:
h_zero_alerts = pd.read_sql_query("""
WITH HABITABILITY_ZERO_RATE AS(
SELECT
    DATE_ADDED,
    CAST(SUM(CASE WHEN HABITABILITY IS 0 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) AS ZERO_RATE
FROM
    HABITABLES
GROUP BY
    DATE_ADDED
)

SELECT
    DATE_ADDED
FROM
    HABITABILITY_ZERO_RATE
WHERE
    ZERO_RATE IS NOT NULL AND
    ZERO_RATE > 0.3
""", conn)
h_zero_alerts = h_zero_alerts \
    .rename(columns={clmn: clmn.lower() for clmn in h_zero_alerts.columns})

In [ ]:
plt.figure(figsize=(20, 10))
plt.bar(all_days, height=h_zero["zero_rate"])
for alert in h_zero_alerts["date_added"]: plt.axvline(x = alert, color = 'r')
plt.show()

In [ ]:
h_zero_alerts["date_added"]

This clearly identifies some problematic timestamps, but it's much too noisy. We wouldn't want a notification for every red line on the above graph. Looking for other (upstream) events can not only prune our alerts, but also help us identify the issue's cause.

See if you can join the timestamps from `h_zero_alerts` with timestamps identifying a `schema_change`.

- *Hint*: try querying the `EXOPLANETS_SCHEMA` table using the same approach from Exercise 2.

In [ ]:
# YOUR CODE HERE
SQL = """

"""
# END YOUR CODE

In [ ]:
joint_anoms = pd.read_sql_query(SQL, conn)
joint_anoms = joint_anoms \
    .rename(columns={clmn: clmn.lower() for clmn in joint_anoms.columns})
joint_anoms

In [ ]:
plt.figure(figsize=(20, 10))
plt.bar(all_days, height=h_zero["zero_rate"])
for alert in joint_anoms["date"]: plt.axvline(x = alert, color = 'r')
plt.show()

You should see a single reported date, `2020-07-19`. Not only have we reduced the number of reports, but we potentially learn something -- that the schema change in `EXOPLANETS` _caused_ the zero rate in `HABITABLES` to spike. By combining data observability pillars, we're one step closer to resolving the problem!

![SegmentLocal](/tree/data/assets/comet.gif "segment")

## 4. Exercise: Diagnosing Another Distribution Issue

Here's another quick mystery. It looks like the `HABITABLES` table returns to normal after a while, if we only look at zero rates. But probing into the **volume** of the table reveals something odd:

In [ ]:
rows_added = pd.read_sql_query("""
SELECT
    DATE_ADDED,
    COUNT(*) AS ROWS_ADDED
FROM
    HABITABLES
GROUP BY
    DATE_ADDED
""", conn)
rows_added = rows_added \
    .rename(columns={clmn: clmn.lower() for clmn in rows_added.columns}) \
    .set_index("date_added") \
    .reindex(all_days)

In [ ]:
plt.figure(figsize=(20, 10))
plt.bar(all_days, height=rows_added["rows_added"])
plt.show()

The row counts added seem to increase by ~1.5x each day starting around `DATE`. We could detect this using a naive threshold:

In [ ]:
h_rc_anoms = pd.read_sql_query("""
WITH ROW_COUNTS AS(
    SELECT
        DATE_ADDED,
        COUNT(*) AS ROWS_ADDED
    FROM
        HABITABLES
    GROUP BY
        DATE_ADDED
)
SELECT
    DATE_ADDED
FROM
    ROW_COUNTS
WHERE
    ROWS_ADDED > 150
""", conn)
h_rc_anoms = h_rc_anoms \
    .rename(columns={clmn: clmn.lower() for clmn in h_rc_anoms.columns})

In [ ]:
plt.figure(figsize=(20, 10))
plt.bar(all_days, height=rows_added["rows_added"])
for alert in h_rc_anoms["date_added"]: plt.axvline(x = alert, color = 'r')
plt.show()

But again, that's not too noisy to be informative. This issue is compounded because volume is usually a problem when it *decreases* (as we saw earlier with freshness). But something must be the cause of this volume change, and turns out, it's a genuine issue.

As another exercise in understanding **distribution**, try querying for _uniqueness_ in `HABITABLES` to find a simultaneous issue.

- *Hint*: What's practically guaranteed to be unique in this table, if anything?

In [ ]:
# YOUR CODE HERE
SQL = """

"""
# END YOUR CODE

In [ ]:
h_uniq = pd.read_sql_query(SQL, conn)
h_uniq = h_uniq.rename(columns={clmn: clmn.lower() for clmn in h_uniq.columns}) \
    .set_index("date_added") \
    .reindex(all_days)

In [ ]:
plt.figure(figsize=(20, 10))
plt.bar(all_days, height=h_uniq["pct_unique"])
plt.show()

A proper query here should reveal something telling -- the `_ID` field in `HABITABLES` is not unique, meaning we may be adding duplicate entries to our table! Semantics should dictate that `_ID` be 100% unique. Try writing a query that turns up the offending dates below:
- As an extra challenge, try filtering all alerts that occur immediately after other ones -- another potential technique for reducing noise.

In [ ]:
# YOUR CODE HERE
SQL = """

"""
# END YOUR CODE

In [ ]:
h_uniq_anoms = pd.read_sql_query(SQL, conn)
h_uniq_anoms = h_uniq_anoms \
    .rename(columns={clmn: clmn.lower() for clmn in h_uniq_anoms.columns})

In [ ]:
h_uniq_anoms

In [ ]:
plt.figure(figsize=(20, 10))
plt.bar(all_days, height=h_uniq["pct_unique"])
for alert in h_uniq_anoms["date_added"]: plt.axvline(x = alert, color = 'r')
plt.show()

# Great work!

This last exercise revealed that certain pillars of data observability are often conjoined to give meaningful alerts (volume and uniqueness; schema change and downstream distributions; etc.). In the next exercise, we'll look at some terms from machine learning to scale our approach.